In [8]:
!pip uninstall pylopt -y

Found existing installation: PyLOpt 0.1.0
Uninstalling PyLOpt-0.1.0:
  Successfully uninstalled PyLOpt-0.1.0


In [10]:
!pip install ../../../artefacts/pylopt-0.1.0-py3-none-any.whl

Processing /home/florianthaler/Documents/research/pylopt/artefacts/pylopt-0.1.0-py3-none-any.whl

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [12]:
import os
import torch
from torch.utils.data import DataLoader
from pathlib import Path

In [4]:
torch.manual_seed(123)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dtype = torch.float32

### Setup training dataset

In [5]:
from pylopt.dataset.ImageDataset import TestImageDataset, TrainingImageDataset

train_data_root_dir = '/home/florianthaler/Documents/data/image_data/BSDS300/images/train'
train_image_dataset = TrainingImageDataset(root_path=train_data_root_dir, dtype=dtype)
batch_size = 128
crop_size = 64

### Setup test dataset

In [6]:
test_data_root_dir = '/home/florianthaler/Documents/data/image_data/some_images'
test_image_dataset = TestImageDataset(root_path=test_data_root_dir, dtype=dtype)

### Setup regulariser

In [9]:
from pylopt.examples.scripts import PRETRAINED_FILTER_MODELS, PRETRAINED_POTENTIAL_MODELS
from pylopt.utils.file_system_utils import create_experiment_dir, get_repo_root_path
from pylopt.regularisers.fields_of_experts.ImageFilter import ImageFilter
from pylopt.regularisers.fields_of_experts.potential import StudentT
from pylopt.regularisers.fields_of_experts.FieldsOfExperts import FieldsOfExperts

repo_root_path = get_repo_root_path(Path().resolve())
image_filter = ImageFilter.from_file(os.path.join(repo_root_path, 
                                                  'data', 'model_data',
                                                  PRETRAINED_FILTER_MODELS['chen-ranftl-pock_2014_scaled_7x7']))

image_filter.freeze()           # freeze image_filter, i.e. parameters are not trainable
potential = StudentT(num_marginals=48, initialisation_mode='uniform', multiplier=0.0005, trainable=True)
regulariser = FieldsOfExperts(potential, image_filter)

ModuleNotFoundError: No module named 'pylopt'

### Setup lower-level solution method

In [14]:
method_lower = 'nag'
options_lower = {'max_num_iterations': 300, 'rel_tol': 1e-5, 'lip_const': 1e5, 'batch_optimisation': False}

### Setup BilevelOptimisation class

In [15]:
from pylopt.bilevel_problem import BilevelOptimisation

path_to_experiment_dir = os.path.join(os.getcwd(), 'experiment')
if not os.path.exists(path_to_experiment_dir):
    os.makedirs(path_to_experiment_dir)

bilevel_optimisation = BilevelOptimisation(method_lower, 
                                           options_lower, 
                                           operator=torch.nn.Identity(),
                                           noise_level=0.1,
                                           differentiation_method='hessian_free',
                                           path_to_experiments_dir=path_to_experiment_dir)

### Initial test loss and psnr

In [10]:
from pylopt.lower_problem import solve_lower

lower_prob_result = solve_lower(energy=energy, method=method, options=options)


### Training loop

In [11]:
lam = 10

### Callbacks and schedulers

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from pylopt.callbacks import SaveModel, PlotFiltersAndPotentials, TrainingMonitor
from pylopt.scheduler import CosineAnnealingLRScheduler

tb_writer = SummaryWriter(log_dir=os.path.join(path_to_eval_dir, 'tensorboard'))
callbacks = [PlotFiltersAndPotentials(test_image_dataset, 
                                      path_to_data_dir=path_to_eval_dir,
                                      plotting_freq=2, 
                                      tb_writer=tb_writer),
             SaveModel(path_to_data_dir=path_to_eval_dir, 
                       save_freq=2),
             TrainingMonitor(test_image_dataset, 
                             config, 
                             method_lower, 
                             options_lower, 
                             l2_loss_func,
                             path_to_eval_dir, 
                             evaluation_freq=1, 
                             tb_writer=tb_writer)
            ]

schedulers = [CosineAnnealingLRScheduler(step_begin=500, 
                                         restart_cycle=None,
                                         step_end=8500,
                                         lr_min=1e-5)]

In [ ]:
method_upper = 'adam'
max_num_iterations = 10000

bilevel_optimisation.learn(regulariser, 
                           lam, 
                           l2_loss_func, 
                           train_image_dataset,
                           optimisation_method_upper=method_upper, 
                           optimisation_options_upper=options_upper,
                           dtype=dtype, device=device, 
                           callbacks=callbacks, 
                           schedulers=schedulers)

### Store trained model and visualisations of training results

In [13]:
save_foe_model(regulariser, os.path.join(path_to_eval_dir, 'models'), model_dir_name='final')

visualise_training_stats(train_loss_list, test_loss_list, psnr_list, evaluation_freq, path_to_eval_dir)
visualise_filter_stats(filters_list, path_to_eval_dir)